# Redis tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [Redis commands](https://redis.io/commands)
* [Redis python client](https://redis-py.readthedocs.io/en/latest/)

Though the tutorial asks you to use Redis commands, they all have an equivalent in the Python client library.

### Import libraries

In [102]:
import redis

In [103]:
r = redis.Redis(host='redis', port=6379)

In [104]:
r.set('foo', 'bar')
value = r.get('foo')
print(value)

b'bar'


You can launch a terminal aside, connect to your server with a Redis client and check that the value is still present :

```bash
vagrant@nosql:~$ redis-cli
127.0.0.1:6379> get foo
"bar"
```

## I. Quick start

### Strings and numbers

**Q** : Store the string `hello world` in key `greetings` with a `SET` command. Then use a `GET` command to retrieve it.

In [105]:
r.set('greetings', 'hello world')
value = r.get('greetings')
print(value)

b'hello world'


**Q** : Create a `connections` key with value 0. Find the `INCR` command to increment its value, then display it. 

Try to increment the value for `greetings`.

In [106]:
r.set("connections", 0)
for _ in range(10):
    r.incr("connections")
value = r.get("connections")
print(value)

b'10'


### Lists

**Q** : Create a `colors` list with strings `red`, `black` and `blue`, using the `RPUSH` command.

In [107]:
r.delete("colors")
lst = ["red","black","blue"]
r.rpush("colors", *lst)

3

**Q** : Display the second element of the list with `LINDEX`

In [108]:
r.lindex("colors", index=1)

b'black'

**Q** : Display the full list with `LRANGE`

In [109]:
r.lrange("colors", start=0, end=-1)

[b'red', b'black', b'blue']

### Using sets

**Q** : Create a `nosql` set containing values `redis`, `mongodb` and `orientdb`, using the `SADD` command.

In [110]:
r.sadd("nosql", *{"redis","mongodb","orientdb"})

0

**Q** : Test if `mysql` is inside the set, using `SISMEMBER`

In [111]:
r.sismember("nosql", value="mysql")

0

**Q** : Add `hbase` to the set with `SADD`, then display all elements of the set with `SMEMBERS`. Try to add `redis` again and see what happens.

In [112]:
r.sadd("nosql", *{"hbase"})
values = r.smembers("nosql")
print(values)

{b'mongodb', b'redis', b'orientdb', b'hbase'}


In [113]:
r.sadd("nosql", *{"redis"})
values = r.smembers("nosql")
print(values)

{b'mongodb', b'redis', b'orientdb', b'hbase'}


### Using sorted sets

Sorted Sets are similar to Redis Sets with the unique feature of values stored in a set. The difference is, every member of a Sorted Set is associated with a score, that is used in order to take the sorted set ordered, from the smallest to the greatest score.

**Q** : Using `ZADD`, create a sorted set `top14` with the following entries :

```
score city
10 Agen 
33 Bordeaux 
32 Brive 
29 Castres 
38 Clermont 
24 Grenoble 
26 La Rochelle 
32 Montpellier 
14 Oyonnax 
20 Pau 
40 Racing 
22 Stade Français 
36 Toulon 
36 Toulouse
```

In [114]:
score = [10, 33, 32, 29, 38, 24, 26, 32, 14, 20, 40, 22, 36, 36]
city = ["Agen", "Bordeaux", "Brive", "Castres", "Clermont", "Grenoble", "La Rochelle", "Montpellier", "Oyonnax", "Pau", "Racing", "Stade Français", "Toulon", "Toulouse"]
for s, c in zip(score, city):
    r.zadd("top14", mapping={c:s})

**Q** : Fetch the score for `Toulon` with `ZSCORE`, and its ranking with `ZRANK`.

In [115]:
zscore = r.zscore("top14", value="Toulon")
zrank = r.zrank("top14", value="Toulon")
print(zscore)
print(zrank)

36.0
10


**Q** : `ZRANK` starts at 0 and scores are sorted from lowest to highest, so we should use the `ZREVRANK` for a true ranking of our cities.

In [116]:
r.zrevrank("top14", value="Toulon", withscore=False)

3

**Q** : Find the commands to display :
* the 3 best teams
* teams with more than 35 points

In [117]:
print(r.zrevrange('top14', 0, 2, withscores=True))
print(r.zrangebyscore("top14", 35,"+Inf"))

[(b'Racing', 40.0), (b'Clermont', 38.0), (b'Toulouse', 36.0)]
[b'Toulon', b'Toulouse', b'Clermont', b'Racing']


### Using dictionaries

**Q** : Create a dictionary `user:1` with `HMSET` with properties `id (1), name (Jean), age (22)`. Display it with `HGETALL`

In [118]:
import json
dico = {
    "id":1,
    "name":"Jean",
    "age":22
}

In [119]:
r.hmset("user:1",dico)

/tmp/ipykernel_22/431901584.py:1: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  r.hmset("user:1",dico)


True

In [120]:
r.hgetall("user:1")

{b'id': b'1', b'name': b'Jean', b'age': b'22', b'city': b'Lyon'}

**Q** : Add a `city (Lyon)` property and rename the user from `Jean` to `Paul`.

In [121]:
r.hset("user:1", "city","Lyon")
r.hset("user:1", "name","Paul")
r.hgetall("user:1")

{b'id': b'1', b'name': b'Paul', b'age': b'22', b'city': b'Lyon'}

## Modelling a query cache with Redis

You are modeling data from a REST query cache system with Redis.
A request is identified by the http method and its url (without the protocol).
The content of the request is stored as it is to be returned on demand.

**Q** : Insert a PUT request in the cache on http://my-api.fr/user/10 whose answer is {"id": 10, "name": "jean"}

In [122]:
http_method = "PUT"
url = "my-api.fr/user/10"  # URL without protocol
response_data = {"id": 10, "name": "Jean"}  # Response content

# Create a unique key for the request using the HTTP method and URL
cache_key = f"{http_method}:{url}"

# Serialize the response data to JSON
response_json = json.dumps(response_data)

# Store the response in Redis with an optional expiration (e.g., 1 hour)
r.set(cache_key, response_json)

True

In [123]:
cached_response = r.get(cache_key)

if cached_response:
    # Deserialize JSON back to Python object
    response_data = json.loads(cached_response)
    print("Cached Response:", response_data)
else:
    print("No cached data found.")

Cached Response: {'id': 10, 'name': 'Jean'}


**Q** : Insert a GET request in the cache on http://my-api.fr/user?size=2 whose answer is[{"id": 10, "name": jean}, {"id": 11, "name": "Claire"}]

In [124]:
http_method = "GET"
url = "my-api.fr/user/10"  # URL without protocol
response_data = [{"id": 10, "name": "Jean"},{"id": 11, "name": "Claire"}]  # Response content

# Create a unique key for the request using the HTTP method and URL
cache_key = f"{http_method}:{url}"

# Serialize the response data to JSON
response_json = json.dumps(response_data)

# Store the response in Redis with an optional expiration (e.g., 1 hour)
r.set(cache_key, response_json)

True

In [125]:
cached_response = r.get(cache_key)

if cached_response:
    # Deserialize JSON back to Python object
    response_data = json.loads(cached_response)
    print("Cached Response:", response_data)
else:
    print("No cached data found.")

Cached Response: [{'id': 10, 'name': 'Jean'}, {'id': 11, 'name': 'Claire'}]


**Q** : Create a set of cache request keys.

In [148]:
request_keys_set = [f"PUT:{url}",f"GET:{url}"]
request_keys_set

['PUT:my-api.fr/user/10', 'GET:my-api.fr/user/10']

In [149]:
r.delete("request_keys_set")
r.sadd("request_keys_set",*request_keys_set)
print(r.smembers("request_keys_set"))

{b'PUT:my-api.fr/user/10', b'GET:my-api.fr/user/10'}


**Q** : Check if the GET request on http: //http://my-api.fr/user

In [150]:
r.get(request_keys_set[0])

b'{"id": 10, "name": "Jean"}'

**Q** : Delete the PUT request on http://my-api.fr/user/10 from the cache.

In [156]:
for r_elem in r.smembers("request_keys_set"):
    if "PUT" in r_elem.decode("utf-8"):
        r.srem("request_keys_set",r_elem)
        break

In [160]:
print(r.smembers("request_keys_set"))

set()


## Postquisites

The folloinwg command removes all data from your Redis cluster.

In [161]:
# !redis-cli flushall
r.flushall()

True